In [1]:
import os 
import re
import torch 
import librosa 
import scipy.signal as sig 
import soundfile as sf
import numpy as np
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, WhisperProcessor, WhisperForConditionalGeneration

In [2]:
# wv2_base_model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")
# wv2_base_proc = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")

# wv2_large_model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-large-960h")
# wv2_large_proc = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h")

In [2]:
whisper_large_model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-large-v3-turbo")
whisper_large_proc = WhisperProcessor.from_pretrained("openai/whisper-large-v3-turbo")

In [3]:
wav_path_kid = '/home/cogsci-lasrlab/Documents/FANN/K1_wav_kid'
wav_path_adult = '/home/cogsci-lasrlab/Documents/FANN/K1_wav_adult'

In [4]:
hotwords = {
    'ring': ['ring', 'wing', 'thumb', 'sun'],
    'wing': ['wing', 'ring', 'sun', 'thumb'],
    'thumb': ['thumb', 'sun', 'ring', 'wing'],
    'sun': ['thumb', 'sun', 'ring', 'wing', 'son'], 
    'fan': ['fan', 'sand', 'red', 'leg'], 
    'sand': ['fan', 'sand', 'red', 'leg'], 
    'red': ['fan', 'sand', 'red', 'leg'], 
    'leg': ['fan', 'sand', 'red', 'leg'], 
    'chip': ['chip', 'ship', 'run', 'one'],
    'ship': ['chip', 'ship', 'run', 'one'],
    'run': ['chip', 'ship', 'run', 'one'],
    'one': ['chip', 'ship', 'run', 'one', 'won'],
    'window': ['window', 'fox', 'socks', 'rainbow'], 
    'rainbow': ['window', 'fox', 'socks', 'rainbow'], 
    'fox': ['window', 'fox', 'socks', 'rainbow', 'sox'], 
    'socks': ['window', 'fox', 'socks', 'rainbow'], 
    'ramp': ['ramp', 'shoe', 'zoo', 'lamp'],
    'lamp': ['ramp', 'shoe', 'zoo', 'lamp'],
    'shoe': ['ramp', 'shoe', 'zoo', 'lamp', 'shoo', 'shue'],
    'zoo': ['ramp', 'shoe', 'zoo', 'lamp'],
    'chick': ['chick', 'sick', 'rock', 'lock'], 
    'sick': ['chick', 'sick', 'rock', 'lock'], 
    'rock': ['chick', 'sick', 'rock', 'lock'], 
    'lock': ['chick', 'sick', 'rock', 'lock', 'loch', 'locke'], 
}

In [5]:
whisper_vocab_dict = whisper_large_proc.tokenizer.get_vocab()
sorted_whisper_vocab_dict = {k.replace('Ġ', ' '): v for k, v in sorted(whisper_vocab_dict.items(), key = lambda item: item[1])}

In [6]:
def clean(text):
    text = re.sub(r'\<\|[^)]*?\|\>', '', text).strip()
    text = re.sub(r'-.!', '', text).strip()
    text = re.sub(r"[^\w\s]", "", text).strip()
    text = text.replace('(', '')
    text = text.replace('\n', '')
    text = text.replace(')', '')
    text = text.replace('?', '')
    text = text.lower()
    text = text.encode('ascii', 'ignore').decode('ascii')
    return text

In [7]:
def extract_target_word(filename):
    parts = filename.split('_')
    target_part = parts[-1]
    target_word = target_part.split('.')[0]
    
    return target_word

In [8]:
from pyctcdecode import build_ctcdecoder

def preprocess(target):
    count = 0 
    new_dict = {}
    for k, v in sorted_whisper_vocab_dict.items():
        for h in hotwords[target]:
            if h in k or h.upper() in k.upper() or h.lower() in k.lower():
                count += 1
                new_dict[k] = v
            
            if len(k) < 2 or '<|' in k:
                count += 1
                new_dict[k] = v
    
    ind_list = [v for v in new_dict.values()]
    decoder = build_ctcdecoder(list(new_dict.keys()), kenlm_model_path = None) 
    
    return ind_list, decoder 

In [ ]:
count = 0 
absolute = 0
confident = 0 

for file_name in os.listdir(wav_path_adult):
    if file_name.endswith('.wav'):
        file_path = os.path.join(wav_path_adult, file_name)
        
        try:
            count += 1
            audio_input, _ = librosa.load(file_path, sr = 16000)
            target = extract_target_word(file_name)
            ind_list, decoder = preprocess(target)
            
            input_values = whisper_large_proc(
                audio_input, 
                sampling_rate = 16000, 
                return_tensors = "pt",
                language = "en"
            ).input_features

            with torch.no_grad():
                predicted_ids = whisper_large_model.generate(
                    input_values, 
                    return_dict_in_generate = True, 
                    output_scores = True
                )

            comp = torch.stack(list(predicted_ids.scores), dim = 0).squeeze()
            logits = torch.exp(comp).cpu().detach().numpy()

            reduced_logits = logits[:, ind_list]

            transcription = decoder.decode(
                reduced_logits, 
                hotwords = hotwords,
                hotword_weight = 100.0
            )

            text = clean(transcription).strip()
            
            new_tries = decoder.decode_beams(
                reduced_logits, 
                hotword_weight = 100.0,
                beam_width = 100,
                token_min_logp = -100000,
                beam_prune_logp = -100000
            )
            
            target_hotwords = hotwords[target]
            h_prob_dict = {h: 0 for h in target_hotwords}

            for k in new_tries:
                if clean(k[0]) in target_hotwords:
                    h_prob_dict[clean(k[0])] += np.exp(k[3])

            print(target, h_prob_dict)
        
        except Exception as e:
            print(f'Error processing file {file_name}: {e}')

Found entries of length > 1 in alphabet. This is unusual unless style is BPE, but the alphabet was not recognized as BPE type. Is this correct?
/home/cogsci-lasrlab/anaconda3/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:818: UserWarning: `return_dict_in_generate` is NOT set to `True`, but `output_scores` is. When `return_dict_in_generate` is not `True`, `output_scores` is ignored.
  warnings.warn(
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_l

thumb {'thumb': 0, 'sun': 0, 'ring': 0, 'wing': 0}


Found entries of length > 1 in alphabet. This is unusual unless style is BPE, but the alphabet was not recognized as BPE type. Is this correct?


red {'fan': 0, 'sand': 0, 'red': 1.00000000000004, 'leg': 0}


Found entries of length > 1 in alphabet. This is unusual unless style is BPE, but the alphabet was not recognized as BPE type. Is this correct?


...

thumb {'thumb': 0.0005247395624449684, 'sun': 0, 'ring': 0, 'wing': 0.0005006197463214405}


Found entries of length > 1 in alphabet. This is unusual unless style is BPE, but the alphabet was not recognized as BPE type. Is this correct?


one {'chip': 0, 'ship': 0, 'run': 0, 'one': 1.00000000000004, 'won': 0}


Found entries of length > 1 in alphabet. This is unusual unless style is BPE, but the alphabet was not recognized as BPE type. Is this correct?


fan {'fan': 1.00000000000004, 'sand': 0, 'red': 0, 'leg': 0}


In [ ]:
# count = 0 
# new_dict = {}
# for k, v in sorted_whisper_vocab_dict.items():
#     for h in hotwords: 
#         if h in k or h.upper() in k.upper() or h.lower() in k.lower():
#             count += 1 
#             new_dict[k] = v 
            
#         if len(k) < 2 or '<|' in k: 
#             count += 1 
#             new_dict[k] = v
            
# print(count)

# ind_list = [v for v in new_dict.values()]

In [ ]:
from pyctcdecode import build_ctcdecoder

decoder = build_ctcdecoder(list(new_dict.keys()), kenlm_model_path=None) 

In [ ]:
audio_input, _ = librosa.load(chip_wav, sr=16000)
input_values = whisper_large_proc(
    audio_input, 
    sampling_rate=16000, 
    return_tensors="pt"
).input_features

with torch.no_grad():
    predicted_ids = whisper_large_model.generate(
        input_values, 
        return_dict_in_generate=True, 
        output_scores=True
    )
    
comp = torch.stack(list(predicted_ids.scores), dim=0).squeeze()
logits = torch.exp(comp).cpu().detach().numpy()

reduced_logits = logits[:, ind_list]

transcription = decoder.decode(
    reduced_logits, 
    hotwords=hotwords,
    hotword_weight=100.0
)

text = clean(transcription).strip()
print(text)

In [ ]:
new_tries = decoder.decode_beams(
    reduced_logits, 
    hotword_weight=100.0,
    beam_width=100,
    token_min_logp=-100000,
    beam_prune_logp=-100000
)

In [ ]:
h_prob_dict = {}
for h in hotwords:
    h_prob_dict[h] = 0
    
for k in new_tries:
    if clean(k[0]) in hotwords:
        h_prob_dict[clean(k[0])] += np.exp(k[3])
        
print(h_prob_dict)